# 🤖 Language-Conditioned RL Agent Training

This notebook trains a language-conditioned agent on BabyAI environments.

## Setup Instructions

1. **Runtime** → **Change runtime type** → Select **T4 GPU**
2. Run all cells in order
3. Download the trained model weights at the end

**Expected Time:**
- BC Training: ~5 minutes
- PPO Training (500 iterations): ~1-2 hours

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install minigrid gymnasium numpy pandas jsonlines pyyaml tqdm python-dotenv -q
!pip install torch transformers accelerate huggingface_hub -q
!pip install 'ray[rllib]' -q

print("\n✅ Dependencies installed!")

## 2. Clone Repository or Upload Code

In [ ]:
# Option A: Clone from GitHub (if you pushed your code)
# !git clone https://github.com/YOUR_USERNAME/Language_Conditioned_Agent_Using_RL.git
# %cd Language_Conditioned_Agent_Using_RL

# Option B: Upload code manually
# Use the file browser on the left to upload your 'src' folder

# For now, let's create the minimal code needed for training:
import os
os.makedirs('src/environment', exist_ok=True)
os.makedirs('src/agents/planner', exist_ok=True)
os.makedirs('src/agents/executor', exist_ok=True)
os.makedirs('src/training', exist_ok=True)
os.makedirs('src/evaluation', exist_ok=True)
os.makedirs('data/demos', exist_ok=True)
os.makedirs('experiments/checkpoints', exist_ok=True)

print("📁 Directory structure created!")

In [ ]:
# Create the minimal environment wrapper
env_wrapper_code = '''
"""MiniGrid wrapper for training."""
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class MiniGridWrapper(gym.Wrapper):
    ACTION_NAMES = ["left", "right", "forward", "pickup", "drop", "toggle", "done"]

    def __init__(self, env_name="BabyAI-GoToObj-v0", max_steps=64, use_dense_reward=True, seed=None):
        import minigrid
        base_env = gym.make(env_name, render_mode="rgb_array")
        if max_steps:
            base_env.unwrapped.max_steps = max_steps
        super().__init__(base_env)
        self.use_dense_reward = use_dense_reward
        self._seed = seed
        self._current_instruction = ""
        self._prev_distance = None

        self.observation_space = spaces.Dict({
            "image": self.env.observation_space["image"],
            "direction": spaces.Discrete(4),
            "instruction": spaces.Text(max_length=256),
        })

    def reset(self, seed=None, options=None):
        reset_seed = seed if seed is not None else self._seed
        obs, info = self.env.reset(seed=reset_seed, options=options)
        self._current_instruction = self.env.unwrapped.mission
        self._prev_distance = None
        return {"image": obs["image"], "direction": obs["direction"], "instruction": self._current_instruction}, info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        if self.use_dense_reward:
            reward = self._shape_reward(reward, terminated)
        return {"image": obs["image"], "direction": obs["direction"], "instruction": self._current_instruction}, reward, terminated, truncated, info

    def _shape_reward(self, reward, terminated):
        if not terminated and hasattr(self.env.unwrapped, "agent_pos"):
            try:
                pos = self.env.unwrapped.agent_pos
                goal = getattr(self.env.unwrapped, "goal_pos", None)
                if goal:
                    dist = abs(pos[0]-goal[0]) + abs(pos[1]-goal[1])
                    if self._prev_distance:
                        reward += 0.01 * (self._prev_distance - dist)
                    self._prev_distance = dist
            except:
                pass
        return reward

    @property
    def instruction(self):
        return self._current_instruction

def make_env(env_name="BabyAI-GoToObj-v0", **kwargs):
    return MiniGridWrapper(env_name, **kwargs)
'''

with open('src/environment/__init__.py', 'w') as f:
    f.write('from src.environment.minigrid_wrapper import MiniGridWrapper, make_env\n')

with open('src/environment/minigrid_wrapper.py', 'w') as f:
    f.write(env_wrapper_code)

print("✅ Environment wrapper created!")

In [ ]:
# Create RLlib environment wrapper
rllib_env_code = '''
"""RLlib environment wrapper."""
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class MiniGridRLlibEnv(gym.Env):
    def __init__(self, config):
        from src.environment.minigrid_wrapper import MiniGridWrapper
        env_name = config.get("env_name", "BabyAI-GoToObj-v0")
        max_steps = config.get("max_steps", 64)
        use_dense_reward = config.get("use_dense_reward", True)
        self.env = MiniGridWrapper(env_name=env_name, max_steps=max_steps, use_dense_reward=use_dense_reward)

        self.observation_space = spaces.Dict({
            "image": spaces.Box(low=0, high=255, shape=(7, 7, 3), dtype=np.float32),
            "direction": spaces.Discrete(4),
            "subgoal": spaces.Box(low=0, high=10, shape=(3,), dtype=np.int32),
        })
        self.action_space = self.env.action_space
        self.current_subgoal = np.array([0, 0, 0], dtype=np.int32)

    def reset(self, *, seed=None, options=None):
        obs, info = self.env.reset(seed=seed, options=options)
        return {"image": obs["image"].astype(np.float32), "direction": obs["direction"], "subgoal": self.current_subgoal}, info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        return {"image": obs["image"].astype(np.float32), "direction": obs["direction"], "subgoal": self.current_subgoal}, reward, terminated, truncated, info

    def render(self):
        return self.env.render()
'''

with open('src/agents/__init__.py', 'w') as f:
    f.write('')
with open('src/agents/executor/__init__.py', 'w') as f:
    f.write('from src.agents.executor.rllib_env import MiniGridRLlibEnv\n')
with open('src/agents/executor/rllib_env.py', 'w') as f:
    f.write(rllib_env_code)

print("✅ RLlib environment wrapper created!")

## 3. Verify Setup

In [ ]:
import sys
sys.path.insert(0, '.')

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Test environment
from src.environment.minigrid_wrapper import MiniGridWrapper
env = MiniGridWrapper("BabyAI-GoToObj-v0")
obs, info = env.reset(seed=42)
print(f"\nEnvironment test:")
print(f"  Instruction: {obs['instruction']}")
print(f"  Image shape: {obs['image'].shape}")

print("\n✅ Setup verified!")

## 4. Generate Expert Demonstrations (Optional BC)

In [ ]:
# Generate demonstrations using BabyAI bot
import gymnasium as gym
import minigrid
import numpy as np
from tqdm import tqdm

def generate_demos(env_name="BabyAI-GoToObj-v0", num_episodes=500, seed=42):
    """Generate expert demonstrations."""
    try:
        from minigrid.utils.baby_ai_bot import BabyAIBot
        has_bot = True
    except ImportError:
        has_bot = False
        print("Warning: BabyAI bot not available, using heuristic policy")

    env = gym.make(env_name)
    demos = []
    np.random.seed(seed)

    for ep in tqdm(range(num_episodes), desc="Generating demos"):
        obs, info = env.reset(seed=seed + ep)
        episode_data = {"obs": [], "actions": [], "instruction": env.unwrapped.mission}

        if has_bot:
            bot = BabyAIBot(env.unwrapped)

        done = False
        while not done:
            episode_data["obs"].append(obs["image"].copy())

            if has_bot:
                try:
                    action = bot.replan()
                    if action is None:
                        action = env.action_space.sample()
                except:
                    action = env.action_space.sample()
            else:
                action = env.action_space.sample()

            episode_data["actions"].append(action)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

        episode_data["success"] = terminated and reward > 0
        if episode_data["success"]:
            demos.append(episode_data)

    env.close()
    print(f"Generated {len(demos)} successful demonstrations")
    return demos

# Generate demos
demos = generate_demos(num_episodes=500)
print(f"\n✅ Generated {len(demos)} demos!")

## 5. PPO Training

In [ ]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from src.agents.executor.rllib_env import MiniGridRLlibEnv

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Configure PPO
config = (
    PPOConfig()
    .environment(
        env=MiniGridRLlibEnv,
        env_config={
            "env_name": "BabyAI-GoToObj-v0",
            "max_steps": 64,
            "use_dense_reward": True,
        },
    )
    .framework("torch")
    .env_runners(
        num_env_runners=2,
        num_envs_per_env_runner=4,
    )
    .training(
        train_batch_size=2048,
        lr=3e-4,
        gamma=0.99,
        clip_param=0.2,
        num_sgd_iter=10,
        entropy_coeff=0.01,
    )
    .resources(num_gpus=1 if torch.cuda.is_available() else 0)
)

print("✅ PPO config created!")

In [ ]:
# Build and train
algo = config.build()
print("✅ PPO algorithm built!")

# Training loop
NUM_ITERATIONS = 200  # Increase for better results (500+ recommended)

results = []
for i in range(NUM_ITERATIONS):
    result = algo.train()

    # Log progress
    reward_mean = result.get("episode_reward_mean", 0)
    episode_len = result.get("episode_len_mean", 0)
    timesteps = result.get("timesteps_total", 0)

    results.append({
        "iteration": i + 1,
        "reward_mean": reward_mean,
        "episode_len": episode_len,
        "timesteps": timesteps,
    })

    if (i + 1) % 10 == 0:
        print(f"Iter {i+1}/{NUM_ITERATIONS}: reward={reward_mean:.2f}, len={episode_len:.1f}, timesteps={timesteps}")

    # Save checkpoint every 50 iterations
    if (i + 1) % 50 == 0:
        checkpoint = algo.save("experiments/checkpoints")
        print(f"  💾 Checkpoint saved: {checkpoint}")

print("\n✅ Training complete!")

In [ ]:
# Plot training curve
import matplotlib.pyplot as plt

iterations = [r["iteration"] for r in results]
rewards = [r["reward_mean"] for r in results]

plt.figure(figsize=(10, 5))
plt.plot(iterations, rewards)
plt.xlabel("Iteration")
plt.ylabel("Mean Episode Reward")
plt.title("PPO Training Curve")
plt.grid(True)
plt.savefig("training_curve.png")
plt.show()

print(f"\nFinal reward: {rewards[-1]:.2f}")

## 6. Evaluate and Save

In [ ]:
# Evaluate the trained agent
def evaluate(algo, num_episodes=100):
    env = MiniGridRLlibEnv({"env_name": "BabyAI-GoToObj-v0", "max_steps": 64})
    successes = 0
    total_reward = 0

    for ep in range(num_episodes):
        obs, info = env.reset(seed=ep)
        done = False
        ep_reward = 0

        while not done:
            action = algo.compute_single_action(obs)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            ep_reward += reward

        if terminated and reward > 0:
            successes += 1
        total_reward += ep_reward

    return {
        "success_rate": successes / num_episodes,
        "mean_reward": total_reward / num_episodes,
    }

eval_results = evaluate(algo, num_episodes=100)
print(f"\n📊 Evaluation Results:")
print(f"   Success Rate: {eval_results['success_rate']:.1%}")
print(f"   Mean Reward:  {eval_results['mean_reward']:.3f}")

In [ ]:
# Save final checkpoint
final_checkpoint = algo.save("experiments/checkpoints/final")
print(f"\n💾 Final checkpoint saved: {final_checkpoint}")

# Zip for download
!zip -r trained_model.zip experiments/checkpoints/
print("\n📦 Model zipped: trained_model.zip")
print("\nDownload this file and extract to your local project!")

In [ ]:
# Download the trained model
from google.colab import files
files.download('trained_model.zip')
files.download('training_curve.png')

In [ ]:
# Cleanup
algo.stop()
ray.shutdown()
print("\n✅ Done! Download your trained model and training curve.")